In [0]:
!pip install autokeras

In [0]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
import numpy as np
from keras import optimizers
from skimage.transform import resize
from autokeras.image.image_supervised import ImageClassifier
import os

Using TensorFlow backend.


In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/out_data_train"

train_data_batch_1   train_data_batch_3  train_data_batch_6  train_data_batch_9
train_data_batch_10  train_data_batch_4  train_data_batch_7  val_data
train_data_batch_2   train_data_batch_5  train_data_batch_8


In [0]:
import pickle
def unpickle(file):
    """load the cifar-10 data"""

    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data

In [0]:
def load_data(data_dir, negatives=False):
    """
    Return train_data, train_labels, test_data, test_labels
    """

    # training data
    train_data = None
    train_labels = []

    # train_data_dict

    for i in range(1, 11):
        train_data_dict = unpickle(data_dir + "/train_data_batch_{}".format(i))
        if i == 1:
            train_data = train_data_dict['data']
        else:
            train_data = np.vstack((train_data, train_data_dict['data']))
#         train_filenames += train_data_dict[b'filenames']
        train_labels += train_data_dict['labels']

    train_data = train_data.reshape((len(train_data), 3, 32, 32))
    print(len(train_data))
    if negatives:
        train_data = cifar_train_data.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        train_data = np.rollaxis(train_data, 1, 4)
#     train_filenames = np.array(train_filenames)
    train_labels = np.array(train_labels)

    # test data
    # cifar_test_data_dict
    # 'batch_label': 'testing batch 1 of 1'
    # 'data': ndarray
    # 'labels': list

    test_data_dict = unpickle(data_dir + "/val_data")
    test_data = test_data_dict['data']
    test_labels = test_data_dict['labels']

    test_data = test_data.reshape((len(test_data), 3, 32, 32))
    if negatives:
        test_data = test_data.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        test_data = np.rollaxis(test_data, 1, 4)
    test_labels = np.array(test_labels)

    return train_data, train_labels, \
           test_data, test_labels

In [0]:
# The data, split between train and test sets:
x_train, y_train, x_test, y_test = \
        load_data('/content/drive/My Drive/out_data_train')
# print('x_train shape:', x_train.shape)
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

100000
x_train shape: (100000, 32, 32, 3)
y_train shape: (100000,)
x_test shape: (2009, 32, 32, 3)
y_test shape: (2009,)


In [0]:
y_train = y_train.reshape(len(y_train),1)
y_test = y_test.reshape(len(y_test),1)
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

x_train shape: (100000, 32, 32, 3)
y_train shape: (100000, 1)
x_test shape: (2009, 32, 32, 3)
y_test shape: (2009, 1)


In [0]:
clf = ImageClassifier(verbose=True, augment=True, searcher_args={'trainer_args':{'max_iter_num':7}})
clf.fit(x_train, y_train)

Saving Directory: /tmp/autokeras_YSDUCR
Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+
                                                                                                    
Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |    16.1410089969635    |  0.11520000000000001   |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|               Training model 1               |
+----------------------------------------------+
                                                                


KeyboardInterrupt: ignored

In [12]:
clf.final_fit(x_train, y_train, x_test, y_test, retrain=False)

Epoch-1, Current Metric - 0:  26%|███████▏                    | 200/782 [01:56<05:53,  1.64 batch/s]

KeyboardInterrupt: ignored

In [11]:
from sklearn.metrics import accuracy_score
y_prediction = clf.predict(x_test)
accuracy_score(y_true=y_test, y_pred=y_prediction)

0.0054753608760577405